In [ ]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

In [ ]:
client = pd.read_sql_table('cliente', engineSource)
client.drop(columns=["nombre_contacto", "ciudad_id", "tipo_cliente_id", "activo", "coordinador_id", "sector", "nit_cliente"], inplace=True)
client.rename(columns={
  "cliente_id": "key_dim_client", 
  "nombre": "name", 
  "direccion": "address",
  "telefono": "phone"
  }, inplace=True)
client["saved"] = pd.to_datetime("now")

client

In [ ]:
client.to_sql("dim_client", engineDestination, index=False, if_exists="replace")